#Install Requirements

In [1]:
!pip install sacrebleu


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 660.6 kB/s eta 0:00:20
     --------------------------------------- 0.0/12.8 MB 393.8 kB/s eta 0:00:33
     --------------------------------------- 0.1/12.8 MB 819.2 kB/s eta 0:00:16
     --------------------------------------- 0.1/12.8 MB 774.0 kB/s eta 0:00:17
      -------------------------------------- 0.2/12.8 MB 841.6 kB/s eta 0:00:15
      -------------------------------------- 0.2/12.8 MB 841.6 kB/s eta 0:00:15
      -------------------------------------- 0.2/12.8 MB 841.6 kB/s eta 0:00:15
      -------------------------------------- 0.3/12.8 MB 682.7 kB/s eta 0:00:19
     - ------------------------------------- 0.4/12.8 MB 859.0 kB/s eta 0:00:15
     - ------------------------------------- 0.5/12.8 MB 942.1 kB/s eta 0:00:14
     - -------------------------------------- 0.6/12.8 MB 1.1 MB/s eta 0:00:11
     -- ------------------------------------- 0.6


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import spacy
nlp_en = spacy.load("en_core_web_sm")

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from scipy.stats import spearmanr
from tqdm.auto import tqdm, trange
from sacrebleu import CHRF
from sklearn.cluster import KMeans
import json
import re
from collections import Counter
import tqdm

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Model (t5)

In [ ]:
t5 = SentenceTransformer('sentence-transformers/sentence-t5-base')

In [ ]:
model = t5

## Import the datasets


In [ ]:
with open("train.model-aware.v2.json", "r") as f:
    trial_data = json.load(f)

train_df_aware = pd.json_normalize(trial_data)

In [ ]:
with open("train.model-agnostic.json", "r") as f:
    trial_data = json.load(f)

train_df_agnostic = pd.json_normalize(trial_data)

In [ ]:
with open("test.model-aware.json", "r") as f:
    trial_data = json.load(f)

test_df_aware = pd.json_normalize(trial_data)

In [ ]:
with open("test.model-agnostic.json", "r") as f:
    trial_data = json.load(f)

test_df_agnostic = pd.json_normalize(trial_data)

In [ ]:
with open("val.model-aware.json", "r") as f:
    trial_data = json.load(f)

val_df_aware = pd.json_normalize(trial_data)

In [ ]:
with open("val.model-agnostic.json", "r") as f:
    trial_data = json.load(f)

val_df_agnostic = pd.json_normalize(trial_data)

# Text Preprocessing

In [ ]:
def extract_word(text):
    match = re.search(r'< define >(.*?)< /define >', text)
    return match.group(1) if match else None

In [ ]:
ar = nlp_en.get_pipe('attribute_ruler')

ar.add([[{"TEXT":"'ve"}]],{"LEMMA":"have"})
ar.add([[{"TEXT":"'m"}]],{"LEMMA":"be"})
ar.add([[{"TEXT":"n't"}]],{"LEMMA":"not"})
ar.add([[{"TEXT":"'ll"}]],{"LEMMA":"will"})
ar.add([[{"TEXT":"'re"}]],{"LEMMA":"be"})

In [ ]:
def preprocess(original_df):

  df = original_df

  df['context'] = ""
  df['word_to_define'] = ""

  for i in tqdm.trange(len(df)):

    df.at[i, 'hyp'] = " ".join(token.lemma_ for token in nlp_en(df.iloc[i]['hyp']) if not token.is_punct)
    df.at[i, 'tgt'] = " ".join(token.lemma_ for token in nlp_en(df.iloc[i]['tgt']) if not token.is_punct)
    if pd.isna(df.at[i, 'src']):
      df.at[i, 'src'] = ''
    else:
      df.at[i, 'src'] = " ".join(token.lemma_ for token in nlp_en(df.iloc[i]['src']) if not token.is_punct)


    hyp = nlp_en(df.at[i, 'hyp'])
    tgt = nlp_en(df.at[i, 'tgt'])
    src = nlp_en(df.at[i, 'src'])

    if df.at[i, 'task'] == 'DM':
      df.at[i, 'word_to_define'] = extract_word(df.at[i, 'src'])
    else:
      df.at[i, 'word_to_define'] = '-'

    hyp_tokens = [token.text for token in hyp]
    tgt_tokens = [token.text for token in tgt]

    if 'model' in df.columns:
      df = pd.get_dummies(df, columns=['model'], prefix='model')
      model_columns = [col for col in df.columns if col.startswith('model_')]
      df[model_columns] = df[model_columns].astype(int)

    intersection = dict(Counter(hyp_tokens) & Counter(tgt_tokens))
    mutual_words = list(intersection.keys())

    df.at[i, 'num_of_common'] = len(mutual_words)

    if df.at[i, 'task'] == 'PG':
      df.at[i, 'context'] = df.at[i, 'src']
    else:
      df.at[i, 'context'] = df.at[i, 'tgt']

  return df


In [ ]:
val_df_aware_processed = preprocess(val_df_aware)

100%|██████████| 501/501 [00:30<00:00, 16.17it/s]


In [ ]:
val_df_agnostic_processed = preprocess(val_df_agnostic)

100%|██████████| 499/499 [00:28<00:00, 17.37it/s]


In [ ]:
test_df_aware_processed = preprocess(test_df_aware)

  0%|          | 0/1500 [00:00<?, ?it/s]

100%|██████████| 1500/1500 [01:32<00:00, 16.17it/s]


In [ ]:
test_df_agnostic_processed = preprocess(test_df_agnostic)

100%|██████████| 1500/1500 [01:23<00:00, 18.05it/s]


In [ ]:
train_df_aware_processed = preprocess(train_df_aware)

100%|██████████| 30000/30000 [33:17<00:00, 15.02it/s]


In [ ]:
train_df_agnostic_processed = preprocess(train_df_agnostic)

  0%|          | 2/30000 [00:00<28:30, 17.54it/s]

100%|██████████| 30000/30000 [23:55<00:00, 20.90it/s]


# Extract Embeddings

In [ ]:
def get_embeddings(df):
  new_df = df

  emb_hyp = model.encode(df.hyp.tolist(), show_progress_bar=True)
  emb_con = model.encode(df.context.tolist(), show_progress_bar=True)

  sims = (emb_con * emb_hyp).sum(1)
  sims.shape

  new_df["full_prob"] = 1-sims
  new_df["sims"] = sims

  return new_df

In [ ]:
train_df_aware = get_embeddings(train_df_aware_processed)

Batches: 100%|██████████| 938/938 [1:12:13<00:00,  4.62s/it]


In [ ]:
train_df_agnostic = get_embeddings(train_df_agnostic_processed)

Batches: 100%|██████████| 938/938 [41:41<00:00,  2.67s/it]  


In [ ]:
test_df_aware = get_embeddings(test_df_aware_processed)

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batches: 100%|██████████| 47/47 [03:37<00:00,  4.62s/it]


In [ ]:
test_df_agnostic = get_embeddings(test_df_agnostic_processed)

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batches: 100%|██████████| 47/47 [01:52<00:00,  2.40s/it]


In [ ]:
val_df_agnostic = get_embeddings(val_df_agnostic_processed)

Batches: 100%|██████████| 16/16 [00:35<00:00,  2.25s/it]


In [ ]:
val_df_aware = get_embeddings(val_df_aware_processed)

Batches: 100%|██████████| 16/16 [00:57<00:00,  3.61s/it]


# Clustering to get labels
if not given (as in val set)

In [ ]:
def get_labels(df):
  model_columns = [col for col in df.columns if col.startswith('model_')]

  if model_columns:
    selected_features = ['sims', 'full_prob'] + model_columns
  else:
    selected_features = ['sims', 'full_prob']


  data = df[selected_features]

  k = 2

  kmeans = KMeans(n_clusters=k, random_state=42)
  df['hallucination'] = kmeans.fit_predict(data)


  print(df['hallucination'].value_counts())

  return df

In [ ]:
train_df_agnostic = get_labels(train_df_agnostic)

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


hallucination
0    20556
1     9444
Name: count, dtype: int64


In [ ]:
train_df_aware = get_labels(train_df_aware)

hallucination
0    20000
1    10000
Name: count, dtype: int64


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Binary encoding of labels in labeled set (val)

In [ ]:
val_df_aware['hallucination'] = np.where(val_df_aware['label'] == 'Hallucination', 1, 0)
val_df_agnostic['hallucination'] = np.where(val_df_agnostic['label'] == 'Hallucination', 1, 0)

# Make Predictions (using classifiers)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
def get_predictions(df_train, df_test):
  y_train = df_train['hallucination']

  x_train = df_train[['sims', 'full_prob']]

  x_test = df_test[['sims', 'full_prob']]

  y_test = None

  # 1) logistic regression =lr
  lr=LogisticRegression(max_iter=5000)
  # 2) random forest =rf
  rf = RandomForestClassifier(n_estimators=30,max_depth=3)
  # 3) GradientBoostingClassifier = gbc
  gbc = GradientBoostingClassifier(n_estimators=100)
  # 4) KNearestNeighbours = knn
  knn = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)
  # 5) CLF = clf
  clf = DecisionTreeClassifier(min_samples_split=10,max_depth=3)

  evc=VotingClassifier(estimators=[('lr',lr),('rf',rf),('gbc',gbc),('knn',knn),('clf',clf)])
  evc.fit(x_train, y_train)

  y_test = evc.predict(x_test)

  df_test["hallucination"] = y_test

  return df_test


Different approaches (labeled/unlabeled training sets)

In [ ]:
#labeled
test_df_aware_pred = get_predictions(val_df_aware, test_df_aware)
test_df_agnostic_pred = get_predictions(val_df_agnostic, test_df_agnostic)

In [ ]:
#unlabeled
test_df_aware_pred = get_predictions(train_df_aware, test_df_aware)
test_df_agnostic_pred = get_predictions(train_df_agnostic, test_df_agnostic)

# Write output files

In [ ]:
def write_output(test_df, output_filename):
    result_list = []

    for _, row in test_df.iterrows():
        current_id = row["id"]

        if row["hallucination"] == 1:
            label = 'Hallucination'
            probability = row["sims"]
        else:
            label = 'Not Hallucination'
            probability = 1 - row["sims"]

        result_dict = {"label": label, "p(Hallucination)": probability,"id": current_id}
        result_list.append(result_dict)

    with open(output_filename, "w") as json_file:
        json.dump(result_list, json_file)

    print(f"Results written to {output_filename}")


In [ ]:
write_output(test_df_aware_pred, 'test.model-aware-output.json')
write_output(test_df_agnostic_pred, 'test.model-agnostic-output.json')

Results written to test.model-aware-output.json
